<a href="https://colab.research.google.com/github/essiesalari/Australian-Celebrities-Face-Recognition-with-Transfer-Learning/blob/main/Australian_Celebrities_Face_Recognition_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Australian Celebrities Face Recognition-VGG16**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [ ]:
! pip install bing_image_downloader

In [ ]:
from bing_image_downloader import downloader
